In [7]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
import time
from torch.utils.data import DataLoader
 
input_path = "../content/DATA_CHAMBER_2021/"


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/DATA_CHAMBER_2021.zip" -d "./"

In [48]:
tensor = transforms.ToTensor()
resise = transforms.Resize(256)
crop_224 = transforms.CenterCrop(224)
crop_64 = transforms.CenterCrop(64)
rotation = transforms.RandomRotation(degrees=20)
blur = transforms.GaussianBlur(kernel_size=3)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


#transforms for train set
transform_train_224 = transforms.Compose([crop_224, tensor])
transform_train_64 = transforms.Compose([crop_64, tensor])

#transforms for test set
transform_test_224 = transforms.Compose([crop_224, tensor])
transform_test_64 = transforms.Compose([crop_64, tensor])

def create_transforms(name = 0):
  if(name == 0):
    transforms_train = transform_train_224
    transforms_test = transform_test_224
  elif (name == 1):
    transforms_train = transform_train_64
    transforms_test = transform_test_64

  return transforms_train, transforms_test
create_tranforms(0)

(Compose(
     CenterCrop(size=(224, 224))
     ToTensor()
 ), Compose(
     CenterCrop(size=(224, 224))
     ToTensor()
 ))

In [25]:
train_dataset_path = input_path + "train"
test_dataset_path = input_path + "test"
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_datasets(transform_train, transform_test):
  trainset = torchvision.datasets.ImageFolder(root = train_dataset_path, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root = test_dataset_path, transform=transform_test)
  return trainset, testset

def prepare_dataloader(trainset, testset):
  trainloader = DataLoader(trainset, batch_size = 8, shuffle = True, num_workers = 2)
  testloader = DataLoader(testset, batch_size = 8, shuffle = False, num_workers = 2)
  return trainloader, testloader

In [26]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)
set_device()

device(type='cpu')

In [15]:
def train_model(model, train_loader, criterion, optimizer, n_epochs):
    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch+1, n_epochs))
        print('-' * 10)
        
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
        
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total
        print('{} loss: {:.4f}, acc: {:.4f}'.format(total, epoch_loss, epoch_acc))

    return model

In [19]:
def creat_model(model_in=None, class_out=1000, device="cpu"):
  if model_in == 'vgg16':  
    model = torchvision.models.vgg16(pretrained=False)
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=class_out, bias=True)
  elif model_in == 'densenet121': 
    model = torchvision.models.densenet121(pretrained=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=class_out, bias=True)
  else:
    pass
    
  model.to(device=device)
  return model


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGG16_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [59]:
def main(model_in):
  classes = get_classes()
  name = [0,1]
  for i in name:
    print(model_in + ' data ' + str(i))
    tranforms_train, tranforms_test = create_transforms(name = i)
    trainset,testset = prepare_datasets(tranforms_train, tranforms_test)
    trainloader, testloader = prepare_dataloader(trainset,testset)
    model = creat_model(model_in=model_in, class_out=len(classes),device = device)
    train_model(model, trainloader, loss_fn, optimizer, 5)


In [ ]:
models = ['vgg16', 'densenet121']

for a in models:
  main(a)

vgg16 data 0
Epoch 1/5
----------


In [ ]:
def test_model(model, test_loader, criterion, optimizer):
    labels_input=list()
    labels_output=list()
    vid_id = list()
        
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        labels_input= labels_input + labels.tolist()
    
        outputs = model(images)
            
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
            
        labels_output= labels_output + preds.tolist()
    return labels_input,labels_output

y_true,y_pred = test_model(VGG16_model, test_loader, loss_fn, optimizer)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(classification_report(y_true,y_pred))
accuracy_score(y_true, y_pred)